In [1]:
import sys
sys.path.insert(0, 'D:\Workspace\workspace_masterarbeit\PainLevelShiftDetection\Classes')

import warnings
warnings.filterwarnings('ignore')

from models import EmbeddingsModel
from trainer import EmbeddingTrainer, all_subjects

from pytorch_metric_learning import distances

from tqdm import tqdm
from functools import partialmethod
tqdm.__init__ = partialmethod(tqdm.__init__, disable=True)

In [2]:
subjects_test = [all_subjects[0]]
subjects_train = [sub for sub in all_subjects if sub not in subjects_test]

### W and B

In [3]:
import wandb
!wandb login

wandb: Currently logged in as: svenk. Use `wandb login --relogin` to force relogin


In [4]:
sweep_config = {
    'method': 'random',
    "early_terminate": {
          "type": "hyperband",
          "max_iter": 10,
          "s": 2
    },
    "metric": {
        'name': 'accuracy',
        'goal': 'maximize'
    },
    "parameters": {
        "lr": {'max': 0.001, 'min': 0.0001},
        "dropout": {'max': 0.6, 'min': 0.4},
        "sigmoid": {"values": [True, False]}
    }
}

In [5]:
def train(config=None):
    with wandb.init(config=config):
        config = wandb.config
        
        trainer_hyperparameter = {
            "path_train": "D:\Workspace\workspace_masterarbeit\PainLevelShiftDetection\FeatureGeneration\dataset_processed\XITE\\normalized_subjects_no_outliner.pkl",
            "subjects_train": subjects_train,
            "path_test": "D:\Workspace\workspace_masterarbeit\PainLevelShiftDetection\FeatureGeneration\dataset_processed\XITE\\normalized_subjects_no_outliner.pkl",
            "subjects_test": subjects_test,
            "acc_tester_metric": "AMI",
            "wandb": True,
            "acc_in_loop": False,
            "learning_rate": config.lr,
            "batch_size": 256,
            "margin": 0.45688,
            "distance": 1,
            "filter": None
        }

        model_hyperparameter = {
            "sigmoid": config.sigmoid,
            "dropout": config.dropout,
            "layers": [207, 512, 256, 64]
        }

        model = EmbeddingsModel(model_hyperparameter)        
        trainer = EmbeddingTrainer(trainer_hyperparameter, model, device="cuda:0")
        trainer.trainloop(5)

In [ ]:
sweep_id = wandb.sweep(sweep_config, project="PainLevelShiftDetection")

In [7]:
wandb.agent(sweep_id, train, count=30)